In [388]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings("ignore")

In [389]:
stop_words = open(os.getcwd() + '/../Data/vietnamese-stopwords.txt', 'r', encoding='utf-8').read()
stop_words = stop_words.split()

In [390]:
url = 'https://phunuvietnam.vn/nhung-nguoi-phu-nu-dat-cuoc-song-vao-che-do-im-lang-ho-thuong-khong-don-gian-512022112114044393.htm'
result = requests.get(url)
c = result.content
soup = BeautifulSoup(c,"lxml")

summary = soup.find('div',class_="content-detail")
summary = summary.find_all('p')
summary = summary[1:-1]
summary = [i.text for i in summary]
summary = ' '.join(summary)
summary

' Thực tế, những người phụ nữ có thể đặt cuộc sống của mình vào "chế độ im lặng" thường không đơn giản, hơn ai hết, họ đều rất hiểu những điều sau đây:\xa0 Không cần mặt trời để chói chang mới là hạnh phúc Có nhiều người đặc biệt thích chia sẻ những bức hình sang chảnh, lộng lẫy của mình lên cho người khác xem, dù ở trong hoàn cảnh nào cũng cố tạo ra trạng thái hoàn hảo để chứng minh cho người ngoài thấy bản thân đang rất hạnh phúc. Sự thật là, trên thế giới này, không ai thực sự quan tâm bạn có hạnh phúc hay không mà luôn có người ghen tị với bạn khi bạn làm tốt và có người làm tổn thương bạn khi bạn làm không tốt.\xa0Bản chất con người rất phức tạp\xa0và cách tốt nhất để bảo vệ chính bạn là không thách thức người khác. Những người phụ nữ thực sự khôn ngoan\xa0biết cách kiềm chế sự phù phiếm của mình. Họ hiểu rằng cuộc sống không phải để người khác nhìn thấy. Hạnh phúc như cốc nước lọc, hãy uống và tự mình biết, không nhất thiết phải phơi bày ra ngoài. Họ chuyển\xa0cuộc sống của mình 

In [391]:
# Load modul 
import os
from gensim.models import KeyedVectors 

w2v = KeyedVectors.load_word2vec_format(os.getcwd() + '/../Data/vi.vec')

In [392]:
from underthesea import sent_tokenize,word_tokenize
#  convert sentence to vector / Hàm chuyển câu thành vector

def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(w2v[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

#  chuyển văn bản thành từng câu / Convert text to sentences
def text2sent(text):
    sentences = [[sentences.lower()] for sentences in sent_tokenize(text)]
    return sentences

#  tính độ tương tự giữa 2 câu / Calculate the similarity between 2 sentences
def similarity(s1, s2):
    return np.dot(s1, s2)

#  tính độ tương tự giữa 1 câu với tất cả các câu khác / Calculate the similarity between 1 sentence and all other sentences
def sim_matrix(sentences):
    n = len(sentences)
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            A[i][j] = similarity(sentences[i], sentences[j])
    return A


In [393]:
#  chuyển văn bản thành từng câu / Convert text to sentences
sentences = text2sent(summary)
sentences

[[' thực tế, những người phụ nữ có thể đặt cuộc sống của mình vào "chế độ im lặng" thường không đơn giản, hơn ai hết, họ đều rất hiểu những điều sau đây:\xa0 không cần mặt trời để chói chang mới là hạnh phúc có nhiều người đặc biệt thích chia sẻ những bức hình sang chảnh, lộng lẫy của mình lên cho người khác xem, dù ở trong hoàn cảnh nào cũng cố tạo ra trạng thái hoàn hảo để chứng minh cho người ngoài thấy bản thân đang rất hạnh phúc.'],
 ['sự thật là, trên thế giới này, không ai thực sự quan tâm bạn có hạnh phúc hay không mà luôn có người ghen tị với bạn khi bạn làm tốt và có người làm tổn thương bạn khi bạn làm không tốt.'],
 ['bản chất con người rất phức tạp\xa0và cách tốt nhất để bảo vệ chính bạn là không thách thức người khác.'],
 ['những người phụ nữ thực sự khôn ngoan\xa0biết cách kiềm chế sự phù phiếm của mình.'],
 ['họ hiểu rằng cuộc sống không phải để người khác nhìn thấy.'],
 ['hạnh phúc như cốc nước lọc, hãy uống và tự mình biết, không nhất thiết phải phơi bày ra ngoài.'],


In [394]:
#  Chuyển câu thành vector / Convert sentences to vector
sentences_vec = [sent2vec(i) for i in sentences]

# Tóm tắt văn bản thông thường theo cách tính trọng số

In [395]:
#  Tính ma trận độ tương tự / Calculate similarity matrix
A = sim_matrix(sentences_vec)

#  Tính độ tương tự trung bình của câu / Calculate the average similarity of the sentence
avg = A.sum(axis=1) / (A.shape[0] - 1)


In [396]:
#  Xây dựng trọng số cho từng câu / Build weight for each sentence
weighted_avg = ((avg - avg.min()) / (avg.max() - avg.min()))
#  Tính điểm cho từng câu / Calculate the score for each sentence
score = (A.dot(weighted_avg)) / weighted_avg.sum()
#  Sắp xếp câu theo thứ tự giảm dần / Sort sentences in descending order
ranked_sentences = sorted(((score[i],s) for i,s in enumerate(sentences_vec)), reverse=True)
    

In [397]:
#  In ra 5 câu có điểm cao nhất / Print the top 5 sentences
for i in range(5):
    print(ranked_sentences[i][1])

[ 4.11238484e-02  1.22845158e-01 -1.31692141e-01 -1.62222367e-02
 -4.16941941e-02 -1.77136436e-02 -8.52024630e-02 -9.61658061e-02
 -3.24561223e-02  1.94812626e-01 -4.56974022e-02  5.48962131e-02
 -4.92873713e-02 -1.58097386e-01 -1.57553121e-01 -2.38210279e-02
 -3.61653343e-02 -5.52021600e-02  6.65742606e-02 -1.70826942e-01
 -4.43071872e-02  8.39349106e-02 -8.49167183e-02  2.62495084e-03
 -1.00982428e-01  6.65790141e-02  4.60896306e-02 -7.60807004e-03
  9.41832215e-02 -2.31427103e-02 -3.49766202e-02  6.06022887e-02
  7.57375592e-03 -4.17144448e-02  3.03093120e-02 -1.28938928e-01
 -4.38637212e-02 -7.58463377e-03 -9.27459747e-02  1.03246653e-02
  5.85571155e-02  8.27160105e-02 -1.10986680e-01 -6.98416084e-02
 -8.02593827e-02 -2.11955398e-01  9.65059772e-02 -2.10440144e-01
 -2.39501558e-02  2.23915372e-02 -1.17677990e-02 -1.12086453e-01
  6.29667044e-02 -1.54151186e-01  2.25538723e-02 -2.23089859e-01
  2.94058886e-03 -7.65654296e-02  5.49421236e-02 -2.04139784e-01
  3.31139229e-02  1.32791

In [398]:
#  Tạo ra văn bản tóm tắt với vector tương ứng / Create a summary text with the corresponding vector
summary_wv = ''
for i in range(5):
    summary_wv += sentences[np.where(sentences_vec == ranked_sentences[i][1])[0][0]][0] + ' '

print(summary_wv)


 thực tế, những người phụ nữ có thể đặt cuộc sống của mình vào "chế độ im lặng" thường không đơn giản, hơn ai hết, họ đều rất hiểu những điều sau đây:  không cần mặt trời để chói chang mới là hạnh phúc có nhiều người đặc biệt thích chia sẻ những bức hình sang chảnh, lộng lẫy của mình lên cho người khác xem, dù ở trong hoàn cảnh nào cũng cố tạo ra trạng thái hoàn hảo để chứng minh cho người ngoài thấy bản thân đang rất hạnh phúc. một số người, không đáng để quan tâm trong cuộc sống, chúng ta ai cũng sẽ gặp phải những người không ưa mình, không thích mình. bớt tranh cãi đi, phớt lờ và mỉm cười trước những lời nói không mấy thiện cảm đó, đừng mang suy nghĩ của họ làm thước đo cho hành động của mình. nhớ rằng, bạn không thể cấm người khác nói hay nghĩ gì nhưng hoàn toàn có quyền lựa chọn những điều mình nghe và nhập vào đầu. nhớ rằng, cho dù một người lựa chọn cuộc sống như thế nào, đi theo con đường ra sao thì đó cũng là cuộc sống của chính họ, không liên quan gì đến người khác. 


# Sử dụng TFIDF và phương pháp cosine

In [399]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#  Tạo ma trận tf-idf / Create tf-idf matrix
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform([s[0] for s in sentences])

#  Tính độ tương tự giữa các câu / Calculate the similarity between sentences
sim_mat = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

#  Tính điểm cho từng câu / Calculate the score for each sentence
sim_scores = sim_mat.sum(axis=0)
ranked_sentences = sorted(((sim_scores[i],s) for i,s in enumerate(sentences)), reverse=True)

#  Chuyển 5 câu có điểm cao nhất thành văn bản tóm tắt / Convert the top 5 sentences to a summary text
summary_tfvec = ''
for i in range(5):
    summary_tfvec += ranked_sentences[i][1][0] + ' '

print(summary_tfvec)



 thực tế, những người phụ nữ có thể đặt cuộc sống của mình vào "chế độ im lặng" thường không đơn giản, hơn ai hết, họ đều rất hiểu những điều sau đây:  không cần mặt trời để chói chang mới là hạnh phúc có nhiều người đặc biệt thích chia sẻ những bức hình sang chảnh, lộng lẫy của mình lên cho người khác xem, dù ở trong hoàn cảnh nào cũng cố tạo ra trạng thái hoàn hảo để chứng minh cho người ngoài thấy bản thân đang rất hạnh phúc. một số người, không đáng để quan tâm trong cuộc sống, chúng ta ai cũng sẽ gặp phải những người không ưa mình, không thích mình. nhớ rằng, cho dù một người lựa chọn cuộc sống như thế nào, đi theo con đường ra sao thì đó cũng là cuộc sống của chính họ, không liên quan gì đến người khác. trong cuộc sống, ai cũng cần học cho mình chữ "nhẫn", học cách kiềm chế cảm xúc của mình và không bị cuốn vào những người cũng như những điều tiêu cực. họ chuyển cuộc sống của mình sang"chế độ im lặng", giữ hạnh phúc cho riêng mình, trân trọng những người xung quanh; dành thời gia

# Sử dụng Count vector thay cho tfidfvector

In [400]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#  Tạo ma trận tf-idf / Create tf-idf matrix
vectorizer = CountVectorizer()
tf_idf_matrix = vectorizer.fit_transform([s[0] for s in sentences])

#  Tính độ tương tự giữa các câu / Calculate the similarity between sentences
sim_mat = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

#  Tính điểm cho từng câu / Calculate the score for each sentence
sim_scores = sim_mat.sum(axis=0)
ranked_sentences = sorted(((sim_scores[i],s) for i,s in enumerate(sentences)), reverse=True)

#  Chuyển 5 câu có điểm cao nhất thành văn bản tóm tắt / Convert the top 5 sentences to a summary text
summary_countvec = ''
for i in range(5):
    summary_countvec += ranked_sentences[i][1][0] + ' '

print(summary_countvec)

 thực tế, những người phụ nữ có thể đặt cuộc sống của mình vào "chế độ im lặng" thường không đơn giản, hơn ai hết, họ đều rất hiểu những điều sau đây:  không cần mặt trời để chói chang mới là hạnh phúc có nhiều người đặc biệt thích chia sẻ những bức hình sang chảnh, lộng lẫy của mình lên cho người khác xem, dù ở trong hoàn cảnh nào cũng cố tạo ra trạng thái hoàn hảo để chứng minh cho người ngoài thấy bản thân đang rất hạnh phúc. một số người, không đáng để quan tâm trong cuộc sống, chúng ta ai cũng sẽ gặp phải những người không ưa mình, không thích mình. họ chuyển cuộc sống của mình sang"chế độ im lặng", giữ hạnh phúc cho riêng mình, trân trọng những người xung quanh; dành thời gian, tình cảm cho những người và những điều quan trọng. trong cuộc sống, ai cũng cần học cho mình chữ "nhẫn", học cách kiềm chế cảm xúc của mình và không bị cuốn vào những người cũng như những điều tiêu cực. gặp phải những lời khó nghe, tiêu cực thì họ sẽ không nghe; không thể bắt người khác im lặng thì bịt chặ

# Sử dụng model kmeans dự đoán

In [401]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

#  Tạo ma trận tf-idf / Create tf-idf matrix
vectorizer_C = CountVectorizer()
tf_idf_matrix_C = vectorizer_C.fit_transform([s[0] for s in sentences])

#  Tạo model KMeans / Create KMeans model
kmeans = KMeans(n_clusters=5, random_state=0).fit(tf_idf_matrix_C)

#  Tìm ra các câu có cùng cluster / Find sentences with the same cluster
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, tf_idf_matrix_C)

#  Chuyển các câu có cùng cluster thành văn bản tóm tắt / Convert sentences with the same cluster to a summary text
summary_KM_C = ''
for i in closest:
    summary_KM_C += sentences[i][0] + ' '

print(summary_KM_C)

một số người, không đáng để quan tâm trong cuộc sống, chúng ta ai cũng sẽ gặp phải những người không ưa mình, không thích mình. sự thật là, trên thế giới này, không ai thực sự quan tâm bạn có hạnh phúc hay không mà luôn có người ghen tị với bạn khi bạn làm tốt và có người làm tổn thương bạn khi bạn làm không tốt. họ hiểu rằng cuộc sống không phải để người khác nhìn thấy.  thực tế, những người phụ nữ có thể đặt cuộc sống của mình vào "chế độ im lặng" thường không đơn giản, hơn ai hết, họ đều rất hiểu những điều sau đây:  không cần mặt trời để chói chang mới là hạnh phúc có nhiều người đặc biệt thích chia sẻ những bức hình sang chảnh, lộng lẫy của mình lên cho người khác xem, dù ở trong hoàn cảnh nào cũng cố tạo ra trạng thái hoàn hảo để chứng minh cho người ngoài thấy bản thân đang rất hạnh phúc. một người phụ nữ thực sự mạnh mẽ biết cách đặt cuộc sống của mình ở "chế độ im lặng". 


In [402]:
#  Tạo ma trận tf-idf / Create tf-idf matrix
vectorizer_T = TfidfVectorizer()
tf_idf_matrix_T = vectorizer_T.fit_transform([s[0] for s in sentences])

#  Tạo model KMeans / Create KMeans model
kmeans = KMeans(n_clusters=5, random_state=0).fit(tf_idf_matrix_T)

#  Tìm ra các câu có cùng cluster / Find sentences with the same cluster
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, tf_idf_matrix_T)

#  Chuyển các câu có cùng cluster thành văn bản tóm tắt / Convert sentences with the same cluster to a summary text
summary_KM_T = ''
for i in closest:
    summary_KM_T += sentences[i][0] + ' '

print(summary_KM_T)


một số người, không đáng để quan tâm trong cuộc sống, chúng ta ai cũng sẽ gặp phải những người không ưa mình, không thích mình. họ chuyển cuộc sống của mình sang"chế độ im lặng", giữ hạnh phúc cho riêng mình, trân trọng những người xung quanh; dành thời gian, tình cảm cho những người và những điều quan trọng. đó là bởi họ biết bản thân mình muốn gì và sẽ hướng đến điều trái tim mình mong đợi thay vì bận lòng bởi việc người khác nghĩ gì về hạnh phúc của mình. sự thật là, trên thế giới này, không ai thực sự quan tâm bạn có hạnh phúc hay không mà luôn có người ghen tị với bạn khi bạn làm tốt và có người làm tổn thương bạn khi bạn làm không tốt. một người phụ nữ thực sự mạnh mẽ biết cách đặt cuộc sống của mình ở "chế độ im lặng". 


In [403]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(sentences_vec)

#  Tìm ra các câu có cùng cluster / Find sentences with the same cluster
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentences_vec)

#  Chuyển các câu có cùng cluster thành văn bản tóm tắt / Convert sentences with the same cluster to a summary text

summary_KM_WV = ''
for i in closest:
    summary_KM_WV += sentences[i][0] + ' '

print(summary_KM_WV)


bớt tranh cãi đi, phớt lờ và mỉm cười trước những lời nói không mấy thiện cảm đó, đừng mang suy nghĩ của họ làm thước đo cho hành động của mình. bản chất con người rất phức tạp và cách tốt nhất để bảo vệ chính bạn là không thách thức người khác.  thực tế, những người phụ nữ có thể đặt cuộc sống của mình vào "chế độ im lặng" thường không đơn giản, hơn ai hết, họ đều rất hiểu những điều sau đây:  không cần mặt trời để chói chang mới là hạnh phúc có nhiều người đặc biệt thích chia sẻ những bức hình sang chảnh, lộng lẫy của mình lên cho người khác xem, dù ở trong hoàn cảnh nào cũng cố tạo ra trạng thái hoàn hảo để chứng minh cho người ngoài thấy bản thân đang rất hạnh phúc. trái tim của họ phong phú hơn, ý thức về giá trị bản thân cũng cao hơn và niềm hạnh phúc đó tỏa ra từ trong ra ngoài. bất kể bạn nói và làm gì, họ cũng sẽ thấy không thuận mắt thuận tai, nói những lời khó nghe về bạn. 


# Thực hiện tóm tắt lại văn bản thành 1 câu với tất cả cá mô hình đã làm

In [404]:
# Chuyển các summeries thành dạng list / Convert summeries to list
summary_list = [summary_wv, summary_tfvec, summary_countvec, summary_KM_C, summary_KM_T, summary_KM_WV]

# Tách các câu trong các summeries / Split sentences in summeries
summary_list = [s.split('. ') for s in summary_list]

# Tạo ma trận tf-idf / Create tf-idf matrix
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform([s[0] for s in sentences])

# Tính độ tương tự giữa các câu / Calculate the similarity between sentences
sim_mat = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

# Tính điểm cho từng câu / Calculate the score for each sentence
sim_scores = sim_mat.sum(axis=0)

# Tạo một list chứa các câu có điểm cao nhất / Create a list containing the sentences with the highest score
ranked_sentences = sorted(((sim_scores[i],s) for i,s in enumerate(sentences)), reverse=True)

# Lấy câu có điểm cao nhất / Get the sentence with the highest score
top_sentence = ranked_sentences[0][1][0]
top_sentence

' thực tế, những người phụ nữ có thể đặt cuộc sống của mình vào "chế độ im lặng" thường không đơn giản, hơn ai hết, họ đều rất hiểu những điều sau đây:\xa0 không cần mặt trời để chói chang mới là hạnh phúc có nhiều người đặc biệt thích chia sẻ những bức hình sang chảnh, lộng lẫy của mình lên cho người khác xem, dù ở trong hoàn cảnh nào cũng cố tạo ra trạng thái hoàn hảo để chứng minh cho người ngoài thấy bản thân đang rất hạnh phúc.'

In [405]:
from underthesea import pos_tag
import string
# Xử lý câu có điểm cao nhất / Process the sentence with the highest score

# lấy các token của câu / get tokens of the sentence
top_sentence_tokens = word_tokenize(top_sentence)

# loại bỏ các token là dấu câu / remove punctuation tokens
top_sentence_tokens = [t for t in top_sentence_tokens if t not in string.punctuation]

# loại bỏ các token là stopword / remove stopword tokens
top_sentence_tokens = [t for t in top_sentence_tokens if t not in stop_words]

# Chuyển thành đoạn văn bản xúc tích / Convert to sentiment text
top_sentence_tokens = ' '.join(top_sentence_tokens)

print('Đoạn văn nói về: ', top_sentence_tokens)

Đoạn văn nói về:  thực tế phụ nữ có thể đặt cuộc sống chế độ im lặng đơn giản hiểu mặt trời chói chang hạnh phúc đặc biệt thích chia sẻ bức hình sang chảnh lộng lẫy xem dù hoàn cảnh cố tạo trạng thái hoàn hảo chứng minh bản thân hạnh phúc
